<a href="https://colab.research.google.com/github/kjaqdenusi/AI/blob/main/HW2/AI_HW_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ---------------------------------------------------------------
# -----------------------  PROBLEM 1  ---------------------------
# bfs — k-th level friends in a social network
# ---------------------------------------------------------------

from collections import deque
from typing import Dict, Set, List, Tuple

# represent a social network as an adjacency list:
#   graph['bob'] == {'pam', 'richard'} means bob <-> pam and bob <-> richard
Graph = Dict[str, Set[str]]

def bfs_kth_level_friends(G: Graph, user: str, k: int) -> Set[str]:
    """
    return the set of nodes at exactly distance k from `user`
    using breadth-first search (bfs)
    """
    if user not in G:                # if user is not in graph return empty set
        return set()

    q = deque([(user, 0)])           # queue holds (node, distance) pairs start with user at distance 0
    visited: Set[str] = {user}       # keeping track of visited nodes so we don’t repeat them
    kth_level: Set[str] = set()      # storing nodes that are exactly distance k here

    while q:                         # (while loop) while there are still nodes to explore
        node, dist = q.popleft()     # take next node from  queue

        if dist == k:                # if this node is at kth level
            kth_level.add(node)      # add it to our results
            continue                 # don’t go deeper from here
        if dist > k:                 # if we already passed k we can stop
            break

        for nbr in G.get(node, set()):   # loop through all neighbors of current node
            if nbr not in visited:       # if we haven’t seen this neighbor yet
                visited.add(nbr)         # mark it visited
                q.append((nbr, dist + 1))# add it to queue at distance +1

    if k == 0:                      # if k is 0 remove user itself (not its own friend)
        kth_level.discard(user)
    return kth_level

# --- test graph for problem 1 ---
G1 = {
    'bob'    : {'pam', 'richard', 'rob'},
    'pam'    : {'bob', 'roger', 'peter'},
    'richard': {'bob'},
    'rob'    : {'bob'},
    'roger'  : {'pam', 'anna'},
    'anna'   : {'roger'},
    'peter'  : {'pam', 'amy'},
    'amy'    : {'peter'},
}

print("=== PROBLEM 1 ===")
print("FindFriends(G1, 'bob', 3):", bfs_kth_level_friends(G1, 'bob', 3))
# expected -> {'amy', 'anna'}

# ---------------------------------------------------------------
# -----------------------  PROBLEM 2  ---------------------------
# bfs — friend-of-friend suggestions (potential friends)
# ---------------------------------------------------------------

def potential_friends(G: Graph, user: str) -> Set[str]:
    """
    returns friends-of-friends (distance exactly 2),
    excluding the user and existing direct friends
    """
    if user not in G:                  # if user not in graph return empty
        return set()

    immediate = set(G[user])           # get users direct friends to exclude later
    q = deque([(user, 0)])             # queue for bfs with (node, distance)
    visited = {user}                   # visited set so we don’t loop forever
    distance_two: Set[str] = set()     # store nodes that end up 2 away

    while q:
        node, dist = q.popleft()

        if dist == 2:                  # if were exactly 2 away
            distance_two.add(node)     # collect it
            continue                   # don’t go further
        if dist > 2:                   # safety--> stop if we go past 2
            break

        for nbr in G.get(node, set()): # go through neighbors
            if nbr not in visited:     # only visit once
                visited.add(nbr)
                q.append((nbr, dist + 1))

    distance_two.discard(user)         # remove user themself
    distance_two -= immediate          # remove any already direct friends
    return distance_two

# --- test graph for problem 2 ---
G2 = {
    'maria' : {'adam', 'sophia', 'maya', 'david'},
    'adam'  : {'maria'},
    'sophia': {'maria', 'maya'},
    'maya'  : {'maria', 'sophia'},
    'david' : {'maria'}
}

print("\n=== PROBLEM 2 ===")
print("PotentialFriends(G2, 'adam') :", potential_friends(G2, 'adam'))    # {'sophia','maya','david'}
print("PotentialFriends(G2, 'david'):", potential_friends(G2, 'david'))   # {'adam','sophia'}
print("PotentialFriends(G2, 'sophia'):", potential_friends(G2, 'sophia')) # {'adam','david'}

# ---------------------------------------------------------------
# -----------------------  PROBLEM 3  ---------------------------
# dfs — path existence in a maze
# ---------------------------------------------------------------

def dfs_reachable(grid: List[List[int]], start: Tuple[int,int], goal: Tuple[int,int]) -> bool:
    """
    depth-first search to check if `goal` is reachable from `start`
    in a 2d maze grid (0=open, 1=wall)
    """
    H, W = len(grid), len(grid[0])     # get height and width
    sy, sx = start                     # starting y and x
    gy, gx = goal                      # goal y and x

    def in_bounds(y, x):               # check if cell is inside grid
        return 0 <= y < H and 0 <= x < W

    if not in_bounds(sy, sx) or not in_bounds(gy, gx):  # invalid start/goal
        return False
    if grid[sy][sx] == 1 or grid[gy][gx] == 1:          # start or goal blocked
        return False

    stack = [(sy, sx)]                 # stack for dfs (lifo)
    visited = set()                    # visited cells

    while stack:
        y, x = stack.pop()             # pop current cell
        if (y, x) in visited:          # skip if already visited
            continue
        visited.add((y, x))            # mark if visited

        if (y, x) == (gy, gx):         # if we reached goal return true
            return True

        # explore directions: up, down, left, right
        for dy, dx in [(-1,0),(1,0),(0,-1),(0,1)]:
            ny, nx = y+dy, x+dx
            if in_bounds(ny, nx) and grid[ny][nx]==0 and (ny,nx) not in visited:
                stack.append((ny, nx))
    return False                       # no path found

# --- test mazes for problem 3 ---
gridA = [
    [0,0,1,0,0,0,0],
    [1,0,1,0,1,1,0],
    [1,0,0,0,0,1,0],
    [1,1,1,1,0,1,0],
    [0,0,0,1,0,0,0],
]
gridB = [
    [0,0,1,0,0,0,0],
    [1,0,1,0,1,1,0],
    [1,0,0,1,0,1,0],  # extra wall blocks path
    [1,1,1,1,0,1,0],
    [0,0,0,1,0,0,0],
]

print("\n=== PROBLEM 3 ===")
print("Maze A:", "SUCCESS" if dfs_reachable(gridA, (0,0), (4,6)) else "FAILURE")
print("Maze B:", "SUCCESS" if dfs_reachable(gridB, (0,0), (4,6)) else "FAILURE")

=== PROBLEM 1 ===
FindFriends(G1, 'bob', 3): {'anna', 'amy'}

=== PROBLEM 2 ===
PotentialFriends(G2, 'adam') : {'sophia', 'maya', 'david'}
PotentialFriends(G2, 'david'): {'sophia', 'maya', 'adam'}
PotentialFriends(G2, 'sophia'): {'adam', 'david'}

=== PROBLEM 3 ===
Maze A: SUCCESS
Maze B: FAILURE
